In [14]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d


import matplotlib.pyplot as plt
from scipy.stats import norm

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# 被験者ID
subject_id = "001"
# 実験番号
experiment_id = "001"

# IVT法のパラメータ
VELOCITY_THRESHOLD = 30 # deg
DURATION_THRESHOLD_MS = 100  # ms

eye_df = pd.read_csv(f"exported_csv/eye_df_id{subject_id}-{experiment_id}.csv")
sampling_df = pd.read_csv(f"exported_csv/sampling_df_id{subject_id}-{experiment_id}.csv")

In [15]:

# モニターサイズ(物理)
monitor_width_cm = 47.6
monitor_height_cm = 26.8
# モニター解像度(px)
monitor_resolution_px = (1920, 1080)
# 視距離(cm)
viewer_distance_cm = 60.0

# cm/pxの変換係数
cm_per_pixel_x = monitor_width_cm / monitor_resolution_px[0]
cm_per_pixel_y = monitor_height_cm / monitor_resolution_px[1]

# ディスプレイ中心を(0, 0)とするための変換
eye_df["gx_centered"] = eye_df["gx"] - 0.5
eye_df["gy_centered"] = eye_df["gy"] - 0.5

# 中心(0,0)での物理距離変換
eye_df["x_cm"] = eye_df["gx_centered"] * monitor_resolution_px[0] * cm_per_pixel_x
eye_df["y_cm"] = eye_df["gy_centered"] * monitor_resolution_px[1] * cm_per_pixel_y

# 視野角の計算
eye_df["x_deg"] = np.degrees(np.arctan2(eye_df["x_cm"], viewer_distance_cm))
eye_df["y_deg"] = np.degrees(np.arctan2(eye_df["y_cm"], viewer_distance_cm))

# データの有効性
eye_df["is_valid"] = eye_df["validity_sum"] > 1

print(eye_df.head())


         gx        gy     epoch_sec        hhmmss  validity_sum  trial  \
0       NaN       NaN  1.732521e+09  16:49:03.491             0     -1   
1  0.476050  0.816155  1.732521e+09  16:49:03.506             2     -1   
2  0.490610  0.818481  1.732521e+09  16:49:03.522             2     -1   
3  0.520527  0.754540  1.732521e+09  16:49:03.538             2     -1   
4  0.507679  0.780874  1.732521e+09  16:49:03.538             2     -1   

   gx_centered  gy_centered      x_cm      y_cm     x_deg     y_deg  is_valid  
0          NaN          NaN       NaN       NaN       NaN       NaN     False  
1    -0.023950     0.316155 -1.140020  8.472954 -1.088508  8.037926      True  
2    -0.009390     0.318481 -0.446964  8.535291 -0.426811  8.096281      True  
3     0.020527     0.254540  0.977085  6.821672  0.932965  6.486364      True  
4     0.007679     0.280874  0.365520  7.527423  0.349042  7.150799      True  


In [16]:
def interpolate_missing(df, time_col="epoch_sec", max_gap_ms=100):
    df= df.copy()
    df["valid"]= df["is_valid"]
    df["interp_x"]= np.nan
    df["interp_y"]= np.nan

    # 有効データを代入
    df.loc[df["valid"], "interp_x"] = df.loc[df["valid"], "x_deg"]
    df.loc[df["valid"], "interp_y"] = df.loc[df["valid"], "y_deg"]
    
    # 内部のみ線形補完
    df["interp_x"] = df["interp_x"].interpolate(limit_area="inside")
    df["interp_y"] = df["interp_y"].interpolate(limit_area="inside")
    
    # 無効区間の連続ブロックを取得
    invalid_mask = ~df["valid"]
    group_id = (invalid_mask != invalid_mask.shift()).cumsum()
    invalid_blocks = df[invalid_mask].groupby(group_id)

    for _, block in invalid_blocks:
        if len(block) == 0:
            continue
        t_start = block[time_col].iloc[0]
        t_end = block[time_col].iloc[-1]
        duration_ms = (t_end - t_start) * 1000
        if duration_ms > max_gap_ms:
            # 100ms超えたら補完結果をNaNに戻す
            df.loc[block.index, ["interp_x", "interp_y"]] = np.nan
    

    return df
    

In [17]:
def apply_gaussian_filter_by_block(df, col_x="interp_x", col_y="interp_y", sigma=1.0):
    df = df.copy()
    df["valid"]= df["is_valid"]
    df["filtered_x"] = np.nan
    df["filtered_y"] = np.nan

    # 有効なデータ（NaNでない）だけを連続ブロックとして抽出
    valid_mask = df[col_x].notna() & df[col_y].notna()
    block_id = (valid_mask != valid_mask.shift()).cumsum()
    blocks = df[valid_mask].groupby(block_id)

    for _, block in blocks:
        idx = block.index
        smoothed_x = gaussian_filter1d(block[col_x], sigma=sigma)
        smoothed_y = gaussian_filter1d(block[col_y], sigma=sigma)
        df.loc[idx, "filtered_x"] = smoothed_x
        df.loc[idx, "filtered_y"] = smoothed_y

    return df


In [18]:
def detect_fixations_ivt(df, velocity_threshold=VELOCITY_THRESHOLD, duration_threshold_ms=DURATION_THRESHOLD_MS):
    fixations = []
    timestamps = df["epoch_sec"].to_numpy()
    xs = df["filtered_x"].to_numpy()
    ys = df["filtered_y"].to_numpy()

In [19]:
new1_df=interpolate_missing(eye_df)

apply_gaussian_filter_by_block(new1_df)


,gx,gy,epoch_sec,hhmmss,validity_sum,trial,gx_centered,gy_centered,x_cm,y_cm,x_deg,y_deg,is_valid,valid,interp_x,interp_y,filtered_x,filtered_y
0,NaN,NaN,1.732521e+09,16:49:03.491,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN
1,0.476050,0.816155,1.732521e+09,16:49:03.506,2,-1,-0.023950,0.316155,-1.140020,8.472954,-1.088508,8.037926,True,True,-1.088508,8.037926,-0.767698,7.960489
2,0.490610,0.818481,1.732521e+09,16:49:03.522,2,-1,-0.009390,0.318481,-0.446964,8.535291,-0.426811,8.096281,True,True,-0.426811,8.096281,-0.244012,7.637767
3,0.520527,0.754540,1.732521e+09,16:49:03.538,2,-1,0.020527,0.254540,0.977085,6.821672,0.932965,6.486364,True,True,0.932965,6.486364,0.362186,7.220752
4,0.507679,0.780874,1.732521e+09,16:49:03.538,2,-1,0.007679,0.280874,0.365520,7.527423,0.349042,7.150799,True,True,0.349042,7.150799,0.719248,7.376814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49784,NaN,NaN,1.732522e+09,17:01:17.593,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN
49785,NaN,NaN,1.732522e+09,17:01:17.593,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN
49786,NaN,NaN,1.732522e+09,17:01:17.593,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN
49787,NaN,NaN,1.732522e+09,17:01:17.593,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN
